### Student Information

Name: 黃有璿

Student ID: 109062505

GitHub ID: anitahuang121

Kaggle name: Anita210503

Kaggle private scoreboard snapshot: 

<img src="pics/KAGGLE.jpg">

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, GRU


import nltk
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem import WordNetLemmatizer 

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import string

# Load data


In [ ]:
import pandas as pd

emotion_data = pd.read_csv("emotion.csv" , sep = "," , names = ["tweet_id" , "emotion"] , skiprows=[0])
identification_data = pd.read_csv("data_identification.csv" , sep = "," , names = ["tweet_id" , "identification"], skiprows=[0])
tweets_data = pd.read_json("tweets_DM.json" , lines = True)

In [ ]:
identification_data.loc[identification_data['identification'] == 'test']

In [ ]:
tweets_data.head()

In [ ]:
import json
from pandas.io.json import json_normalize

data_df = json_normalize(tweets_data._source)


In [ ]:
data_df = data_df.reindex(columns=['tweet.tweet_id','tweet.hashtags','tweet.text'])
data_df = data_df.rename(columns={'tweet.tweet_id':'tweet_id' , 'tweet.hashtags':'hashtags' , 'tweet.text':'text' })

In [ ]:
data_merge_emotion = pd.merge(data_df, emotion_data , on=['tweet_id'], how="outer")

In [ ]:
data_all_info = pd.merge(data_merge_emotion, identification_data , on=['tweet_id'], how="outer")

In [ ]:
[(data_all_info['identification'] == 'train') & (data_all_info['emotion'].isnull())]

In [ ]:
grouped = data_all_info.groupby(data_all_info.identification) 
train_df = grouped.get_group("train") 
test_df = grouped.get_group("test") 

In [ ]:
## save to pickle file

train_df.to_pickle("train_df.pkl") 
test_df.to_pickle("test_df.pkl")


# Preprocessing

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import CSVLogger
import nltk
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np

## load a pickle file
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")


remove punctuation, <LH>, numbers, stopwords
    
    
stemming and lemmatization

In [ ]:
import string
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

train_df['text_without_punct'] = train_df['text'].apply(lambda x : remove_punctuation(x))
test_df['text_without_punct'] = test_df['text'].apply(lambda x : remove_punctuation(x))


In [ ]:
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer("[\w']+")
filt = '[!"#$%&()*,-./:;<=>?@[\]^_`{|}~ ]'

train_df['split'] = train_df['text'] .apply(lambda x: x.replace("<LH>",""))
train_df['split'] = train_df['split'].apply(lambda x: re.sub(filt, ' ', x.lower()))
train_df['split'] = train_df['split'].apply(lambda x: re.sub(r'\d+', " ", x))
train_df['split'] = train_df['split'].apply(lambda x: re.sub('<[^<]+?>', " ", x))
train_df['split'] = train_df['split'].apply(lambda x: x.split(" "))

test_df['split'] = test_df['text'] .apply(lambda x: x.replace("<LH>",""))
test_df['split'] = test_df['split'].apply(lambda x: re.sub(filt, ' ', x.lower()))
test_df['split'] = test_df['split'].apply(lambda x: re.sub(r'\d+', " ", x))
test_df['split'] = test_df['split'].apply(lambda x: re.sub('<[^<]+?>', " ", x))
test_df['split'] = test_df['split'].apply(lambda x: x.split(" "))


In [ ]:
stop_words = set(stopwords.words('english'))
train_df['no_stopwords'] = train_df['split'].apply(lambda x: [item for item in x if item not in stop_words])
train_df['no_stopwords'] = train_df['no_stopwords'].apply(lambda x: [item for item in x if len(item) > 1])
test_df['no_stopwords'] = test_df['split'].apply(lambda x: [item for item in x if item not in stop_words])
test_df['no_stopwords'] = test_df['no_stopwords'].apply(lambda x: [item for item in x if len(item) > 1])

In [ ]:
porter = nltk.stem.PorterStemmer()
lemmatizer = WordNetLemmatizer() 
train_df['stemmed'] = train_df['no_stopwords'].apply(lambda x: [porter.stem(y) for y in x])
test_df['stemmed'] = test_df['no_stopwords'].apply(lambda x: [porter.stem(y) for y in x])
train_df['lemmatization'] = train_df['no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
test_df['lemmatization'] = test_df['no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
train_df['lem_ste'] = train_df['lemmatization'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
test_df['lem_ste'] = test_df['lemmatization'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [ ]:
train_df['clean'] = train_df['lem_ste'].apply(lambda x: [item for item in x if not item.isnumeric() ])
test_df['clean'] = test_df['lem_ste'].apply(lambda x: [item for item in x if not item.isnumeric() ])

In [ ]:
train_df['cleaned'] = train_df['clean'].apply(' '.join)
test_df['cleaned'] = test_df['clean'].apply(' '.join)


In [ ]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=15000, 
                             use_idf=True,  
                             tokenizer=tweet_tokenizer.tokenize,
                             stop_words=stopwords.words('english').append('lh')
                             )

tfidf_vectorizer.fit(train_df['cleaned'])

In [ ]:
print(tfidf_vectorizer.get_feature_names()[:100])
print(tfidf_vectorizer.get_feature_names()[-100:])

In [ ]:
#save cleaned data
train_df.to_pickle("train_cleaned_df.pkl") 
test_df.to_pickle("test_cleaned_df.pkl")

# Load cleaned data

In [3]:
train_df = pd.read_pickle("train_cleaned_df.pkl") 
test_df = pd.read_pickle("test_cleaned_df.pkl")

In [25]:
train_df

,hashtags,tweet_id,text,emotion,identification,text_without_punct,split,no_stopwords,stemmed,lemmatization,lem_ste,clean,cleaned
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation,train,People who post add me on Snapchat must be deh...,"[people, who, post, , add, me, on, , snapchat,...","[people, post, add, snapchat, must, dehydrated...","[peopl, post, add, snapchat, must, dehydr, cuz...","[people, post, add, snapchat, must, dehydrated...","[people, post, add, snapchat, must, dehydrated...","[people, post, add, snapchat, must, dehydrated...",people post add snapchat must dehydrated cuz m...
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness,train,brianklaas As we see Trump is dangerous to fre...,"[, brianklaas, as, we, see, , trump, is, dange...","[brianklaas, see, trump, dangerous, freepress,...","[brianklaa, see, trump, danger, freepress, aro...","[brianklaas, see, trump, dangerous, freepress,...","[brianklaas, see, trump, dangerous, freepress,...","[brianklaas, see, trump, dangerous, freepress,...",brianklaas see trump dangerous freepress aroun...
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,train,Now ISSA is stalking Tasha 😂😂😂 LH,"[now, issa, is, stalking, tasha, 😂😂😂, ]","[issa, stalking, tasha, 😂😂😂]","[issa, stalk, tasha, 😂😂😂]","[issa, stalking, tasha, 😂😂😂]","[issa, stalking, tasha, 😂😂😂]","[issa, stalking, tasha, 😂😂😂]",issa stalking tasha 😂😂😂
5,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,train,RISKshow TheKevinAllison Thx for the BEST TIME...,"[, riskshow, , thekevinallison, thx, for, the,...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...","[riskshow, thekevinallison, thx, best, time, t...",riskshow thekevinallison thx best time tonight...
6,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation,train,Still waiting on those supplies Liscus LH,"[still, waiting, on, those, supplies, liscus, , ]","[still, waiting, supplies, liscus]","[still, wait, suppli, liscu]","[still, waiting, supply, liscus]","[still, waiting, supply, liscus]","[still, waiting, supply, liscus]",still waiting supply liscus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867526,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy,train,Im SO HAPPY NoWonder the name of this show Hap...,"[i'm, so, happy, , , , , nowonder, the, name, ...","[i'm, happy, nowonder, name, show, happy, happ...","[i'm, happi, nowond, name, show, happi, happys...","[i'm, happy, nowonder, name, show, happy, happ...","[i'm, happy, nowonder, name, show, happy, happ...","[i'm, happy, nowonder, name, show, happy, happ...",i'm happy nowonder name show happy happysyfy s...
1867527,[],0x38959e,In every circumtance I'd like to be thankful t...,joy,train,In every circumtance Id like to be thankful to...,"[in, every, circumtance, i'd, like, to, be, th...","[every, circumtance, i'd, like, thankful, almi...","[everi, circumt, i'd, like, thank, almighti, j...","[every, circumtance, i'd, like, thankful, almi...","[every, circumtance, i'd, like, thankful, almi...","[every, circumtance, i'd, like, thankful, almi...",every circumtance i'd like thankful almighty j...
1867528,[blessyou],0x2cbca6,there's currently two girls walking around the...,joy,train,theres currently two girls walking around the ...,"[there's, currently, two, girls, walking, arou...","[there's, currently, two, girls, walking, arou...","[there', current, two, girl, walk, around, lib...","[there's, currently, two, girl, walking, aroun...","[there's, currently, two, girl, walking, aroun...","[there's, currently, two, girl, walking, aroun...",there's currently two girl walking around libr...
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",joy,train,Ah corp

In [4]:
tweet_tokenizer = TweetTokenizer()

tfidf_vectorizer = TfidfVectorizer(max_features=15000, 
                             use_idf=True,  
                             tokenizer=tweet_tokenizer.tokenize,
                             stop_words=stopwords.words('english')
                             )

tfidf_vectorizer.fit(train_df['cleaned'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(max_features=15000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f00021fa3c8>>)

In [5]:
print(tfidf_vectorizer.get_feature_names()[:100])
print(tfidf_vectorizer.get_feature_names()[-100:])

["'", '+', '\\', "a's", 'aa', 'aaa', 'aajtak', 'aamir', 'aap', 'aaron', 'aaroncarter', 'aaronrodgers', 'aarti', 'ab', 'abandon', 'abandoned', 'abba', 'abbey', 'abby', 'abc', 'abcnetwork', 'abcnews', 'abcpolitics', 'aberdeen', 'abhlatino', 'abi', 'abide', 'abiding', 'ability', 'able', 'aboard', 'abominable', 'abomination', 'abortion', 'abound', 'aboveonly', 'abraham', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absorbed', 'absurd', 'abt', 'abu', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abuser', 'abusing', 'abusive', 'abyss', 'ac', 'aca', 'academic', 'academy', 'acc', 'accent', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessible', 'accessory', 'accident', 'accidentally', 'accomplish', 'accomplished', 'accomplishing', 'accomplishment', 'accord', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'acct', 'accuracy', 'accurate', 'accusation', 'accuse', 'accused', 'acc

# Deal with categorical label(y)

In [6]:
from tensorflow.keras.utils import to_categorical
emotion_array = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise','trust']
def emotion_to_sequence(emotions):
    tokenized_emotions = []
    for string in emotions:
        token = emotion_array.index(string)
        tokenized_emotions.append(token)
    return to_categorical(tokenized_emotions)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(emotion_array)

print('check label: ', label_encoder.classes_)
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)
def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']


# Embedding

In [8]:
Dictionary = {}
for idx, token in enumerate(tfidf_vectorizer.get_feature_names()):
    Dictionary[token] = idx

In [9]:
from gensim.models import KeyedVectors
model_path = "GoogleNews/GoogleNews-vectors-negative300.bin.gz"
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [10]:
EMBEDDING_DIM = 300
def word2vec_matrix():
    matrix_emb = np.zeros((len(Dictionary)+1, EMBEDDING_DIM)) # replace MAX_WORD to len(Dictionary)
    for word, idx in Dictionary.items():
        try:
            vector =  word2vec_model.wv[word]
        except:
            vector = np.zeros(300,)
        if idx < 15000:
            matrix_emb[idx] = np.array(vector)
    matrix_emb[15000] = np.zeros(300,)     
    return matrix_emb

In [11]:
MAX_SEQUENCE_LENGTH = 50

def text_to_sequence(texts):
    tokenized_texts = []
    for string in texts:
        tokenized_text = []
        for idx, word in enumerate(string.split(' ')):
            
            # truncate
            if idx >= MAX_SEQUENCE_LENGTH: break 
                
            try:
                token = token_dict[word.lower()]
            except:
                token = 10000
            tokenized_text.append(token)
            
        # padding 
        if len(tokenized_text) < MAX_SEQUENCE_LENGTH: 
            tokenized_text.extend([10000]*(MAX_SEQUENCE_LENGTH-len(tokenized_text)))
                
        tokenized_texts.append(tokenized_text)

    return np.array(tokenized_texts)

In [12]:
train_df = train_df.sample(frac=1)
train_sequence = text_to_sequence(train_df['cleaned'])
train_label = emotion_to_sequence(train_df['emotion'])

test_sequence = text_to_sequence(test_df['cleaned'])


In [20]:
MAX_WORD = 15000
LSTM1_DIM = 256
DENSE1_DIM = 512
DENSE2_DIM = 64
CATEGORY_NUM = 8
BATCH_SIZE = 1000
VALIDATION_SPLIT = 0.2
EPOCHS = 6

csv_logger = CSVLogger('logs/training_log_LSTM.csv')
                    

def model_setting(texts, labels, matrix_emb="uniform"):    
    DROPOUT = 0.2
    
    if matrix_emb != "uniform": matrix_emb = Constant(matrix_emb)
    
#     split data to train & validation
    texts_train, texts_val, labels_train, labels_val = train_test_split(texts, labels, test_size=0.2, random_state=42)
    
    # model build
    model = Sequential()
    model.add(Embedding(MAX_WORD+1, EMBEDDING_DIM, embeddings_initializer=matrix_emb, input_length=MAX_SEQUENCE_LENGTH, trainable=True)) # with embedding matrix
    model.add(LSTM(LSTM1_DIM, dropout=DROPOUT, recurrent_dropout=DROPOUT))
    model.add(Dense(DENSE1_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dense(DENSE2_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dense(CATEGORY_NUM, kernel_initializer='normal', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
    model.summary()

    
    history_callback = model.fit(
        texts_train, 
        labels_train, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        callbacks=[csv_logger],
        verbose=1, 
        shuffle=True, 
        validation_data=(texts_val, labels_val))
    
    pred_result = model.predict(test_sequence, batch_size=1000)
    
    pred_result_val = model.predict(texts_val, batch_size=1000)

    return history_callback, pred_result, labels_val, pred_result_val

In [ ]:
history, pred_result, labels_val, pred_result_val  = model_setting(train_sequence, train_label, word2vec_matrix())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  app.launch_new_instance()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 300)           4500300   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
Total params: 5,235,604
Trainable params: 5,235,604
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [ ]:
pred_result_val[:5]

In [ ]:
pred_result_val = label_decode(label_encoder, pred_result_val)
pred_result_val[:5]

In [ ]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, labels_val), pred_result_val), 2)))

In [ ]:
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log_LSTM.csv")
training_log

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train accuracy', 'Val accuracy'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Val loss'], loc='upper left')
plt.show()

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

pred_form['emotion'] = label_decode(label_encoder, pred_result)
pred_form.to_csv('prediction_lstm.csv', index=False)

# Other method I've tried

# METHOD1： DECISION TREE
the result is not good, the best accuracy = 0.32959

In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
tfidf_vectorizer.fit(train_df['text'])

train_data_TFIDF_features_10000 = tfidf_vectorizer.transform(train_df['text'])

## check dimension
train_data_TFIDF_features_10000.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = tfidf_vectorizer.transform(train_df['text'])
y_train = train_df['emotion']

X_test = tfidf_vectorizer.transform(test_df['text'])
y_test = test_df['emotion']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)


In [ ]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

In [ ]:
pred_form['emotion'] = y_test_pred
pred_form.to_csv('prediction.csv', index=False)

# METHOD2： DEEP LEARNING
This model has accuracy 0.42

### Prepare data (X, y)


In [ ]:
train_df = pd.read_pickle("train_cleaned_df.pkl")
test_df = pd.read_pickle("test_cleaned_df.pkl")
train_df = train_df.sample(frac=1)

In [ ]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=15000, 
                             use_idf=True,  
                             tokenizer=tweet_tokenizer.tokenize,
                             stop_words=stopwords.words('english').append('lh')
                             )

tfidf_vectorizer.fit(train_df['cleaned'])data_df

In [ ]:
sep = int(train_df.shape[0] * 0.8) # 20% for validation

train_dataset_X = train_df[:sep]
train_dataset_y = train_df.emotion[:sep]
validation_dataset_X = train_df[sep:]
validation_dataset_y = train_df.emotion[sep:]

all_trainset_X = train_df
all_trainset_y = train_df.emotion

print(train_dataset_X.shape)
print(train_dataset_y.shape)
print(validation_dataset_X.shape)
print(validation_dataset_y.shape)

In [ ]:
import keras

# standardize name (X, y) 
X_train = tfidf_vectorizer.transform(train_dataset_X['cleaned'])
y_train = train_dataset_y

X_val = tfidf_vectorizer.transform(validation_dataset_X['cleaned'])
y_val = validation_dataset_y

all_X_train =  tfidf_vectorizer.transform(all_trainset_X['cleaned'])
all_y_train = all_trainset_y

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

In [ ]:
test_dataset_X = tfidf_vectorizer.transform(test_df['cleaned'])

In [ ]:
print('x_test_data.shape: ', test_dataset_X.shape)

### Deal with categorical label (y)

Rather than put your label `train_df['emotion']` directly into a model, we have to process these categorical (or say nominal) label by ourselves. 

Here, we use the basic method [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) to transform our categorical  labels to numerical ones.


In [ ]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_val.shape: ', y_val.shape)



In [ ]:
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_val = label_encode(label_encoder, y_val)
all_y_train = label_encode(label_encoder, all_y_train)

print('\n\n## After convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_val.shape: ', y_val.shape)
print('all_y_train.shape: ', all_y_train.shape)

### BUILD MODEL


In [ ]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ReLU, Softmax, Dense

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=256)(X)  # 512
H1 = ReLU()(X_W1)

# 1st hidden layer
H1_W2 = Dense(units=64)(H1)  # 256
H2 = ReLU()(H1_W2)

# 2nd hidden layer
H2_W3 = Dense(units=32)(H2)  # 64
H3 = ReLU()(H2_W3)


# 2nd hidden layer
H3_W4 = Dense(units=8)(H3)  # 32
H4 = Softmax()(H3_W4)

# output layer
#H4_W5 = Dense(units=output_shape)(H4)  # 8
#H5 = Softmax()(H4_W5)

model_output = H4

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

### TRAIN

In [ ]:
csv_logger = CSVLogger('logs/training_log_kaggle.csv')

# training setting
epochs = 5
batch_size = 1000

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_val, y_val)
                    )
print('training finish')

In [ ]:
## predict
pred_result_val = model.predict(X_val, batch_size=1000)
pred_result_val[:5]

In [ ]:
pred_result_val = label_decode(label_encoder, pred_result_val)
pred_result_val[:5]

In [ ]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_val), pred_result_val), 2)))

In [ ]:
#Let's take a look at the training log
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log.csv")
training_log

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train accuracy', 'Val accuracy'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Val loss'], loc='upper left')
plt.show()

In [ ]:
pred_form = pd.read_pickle("test_df.pkl")
pred_form = pred_form.drop('hashtags', axis=1)
pred_form = pred_form.drop('text', axis=1)
pred_form = pred_form.drop('identification', axis=1)
pred_form.rename(columns={'tweet_id':'id'}, inplace=True)

In [ ]:
y_test_dense = model.predict(test_dataset_X)
y_test_dense = label_decode(label_encoder, y_test_dense)
pred_form['emotion'] = y_test_dense
pred_form.to_csv('prediction_dense.csv', index=False)